## Step 1: Open dataset and only select those faces for which we have 70 or more images.

## Step 2: Print a few of the faces to familiarized yourself with the data.

## Step 3: Graph the different labels vs their count.

## Step 4: Notice that the number of features in our dataset is fairly large. This is a good moment to apply PCA to reduce the dimensionality of our dataset. Lets choose 150 components.

## Step 5: A really cool thing about PCA is that it lets you compute the mean of each entry which we can then use to obtain the 'average' face in our dataset.

## Step 6: Plot the components of the PCA. Notice that these are always ordered by importance.

# Exercise 2:

### Logistic Regression

### kNeighbors Classifier

### Linear Discriminant

### Naive Bayes

### Part 2: Which one had the best performance? Which one had the worst performance?

### Part 3: Any idea why the score on the top two differs so drastically from the last two?

### Part 4: Find the log_loss, precision, recall, f_score of the best model.

### Part 5: Plot the Confusion Matrix of the best model.

## Part 6 (optional): Edit the code from *Step 2* to display not only the image but also the label and color code the label in red if your model got it wrong or black if it got it right.